In [1]:
import plotly.express as px
import pandas as pd
import plotly.offline as pyo
import dash
from dash import dcc
from dash import html
import plotly.graph_objects as go

In [2]:
def color_scale(value):
    if value < 50:
        return "#FF0000"#red
    elif value >= 50 and value <= 80:
        return "#FFFF00" #yellow
    elif value > 80:
        return "#00FF00" #green

In [3]:
df=pd.read_csv('DataModelDetailedwDataTypes.csv')

In [4]:
#dfmean=df.groupby(['Country'])['Coverage'].mean()
#dfmean.to_csv("avgpercountry.csv", index=True)

In [5]:
df['color']=df['Coverage'].apply(color_scale)
df['gap']=100-df['Coverage']
#print(df.columns)
#print(df[['Coverage', 'gap']].head())
fig_coverage=px.sunburst(df, color='color',path=['Region','Country','Brand','Model','Volume','Coverage'],
                hover_name='Model',
                #hover_data={'Columnname':False}, # removes a column from the hover data
                maxdepth=3,title='Click in the chart to explore our data coverage', 
                #template='ggplot2',
                branchvalues='total', 
                values='Volume'
               )
fig_coverage.update_traces(textinfo='label+ percent entry')
#px.sunburst()
pyo.plot(fig_coverage, auto_open=True, output_type='file', filename='sunburst.html')

'sunburst.html'

In [7]:
fig_gap=px.sunburst(df, color='color',path=['Region','Country','Brand','Model','gap'],
                hover_name='Model',
                #hover_data={'Columnname':False}, # removes a column from the hover data
                maxdepth=3,title='Click in the chart to explore the GAPS in our data coverage', 
                #template='ggplot2',
                branchvalues='remainder', #branchvalues='total',
                values='gap'
               )
fig_gap.update_traces(textinfo='label')
#px.sunburst()
pyo.plot(fig_gap, auto_open=True, output_type='file', filename='GAP_sunburst.html')

'GAP_sunburst.html'

In [8]:
# Define tables
table_coverage = dcc.Graph(id='table-coverage')
table_gap = dcc.Graph(id='table-gap')

In [9]:
# Define slider
slider_years = dcc.RangeSlider(
    id='slider-years',
    min=df['Year'].min(),
    max=df['Year'].max(),
    value=[df['Year'].min(), df['Year'].max()],
    marks={str(year): str(year) for year in df['Year'].unique()},
    step=None
)

In [10]:
# Define app layout
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Coverage View', children=[
            html.Div([
                html.Div([dcc.Graph(id='sunburst-coverage', figure=fig_coverage)], className='six columns'),
                html.Div([table_coverage], className='six columns'),
            ], className='row'),
        ]),
        dcc.Tab(label='Planner View', children=[
            html.Div([
                html.Div([dcc.Graph(id='sunburst-gap', figure=fig_gap)], className='six columns'),
                html.Div([table_gap], className='six columns'),
            ], className='row'),
        ]),
    ]),
    html.Div(slider_years),
])

In [12]:
#Define callback for coverage table
@app.callback(
    dash.dependencies.Output('table-coverage', 'figure'),
    [dash.dependencies.Input('sunburst-coverage', 'clickData'),
     dash.dependencies.Input('slider-years', 'value')])
def update_table_coverage(click_data, year_range):
    if click_data is None:
        df_filtered = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1])]
    else:
        model_name = click_data['points'][0]['hovertext']
        df_filtered = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1]) & (df['Model'] == model_name)]
    fig = go.table(df_filtered, height=400)
    return fig

In [ ]:
#color_scale=[[0, "red"], [0.60, "yellow"],[0.8, "green"]]
#dfmap['Coverage_in_percentage'] = dfmap['Coverage'].astype(str) + '%'
#px.choropleth(dfmap, locations='Country',color= 'color', title='Coverage in Europe',hover_data=['Coverage_in_percentage'],scope='europe')

In [ ]:
#import plotly.express as px
#import numpy as np
#df = px.data.gapminder().query("year == 2007")
#fig = px.sunburst(df, path=['continent', 'country'], values='pop',
#                  color='lifeExp', hover_data=['iso_alpha'],
#                  color_continuous_scale='RdBu',
#                  color_continuous_midpoint=np.average(df['lifeExp'], weights=df['pop']))
#fig.show()

In [ ]:
#This code successfuly creates a european map with a trafic light color scale but it is continuous
#dfmap = pd.read_csv('DataModelCoverageforMap.csv')
#dfmap['color']=dfmap['Coverage'].apply(color_scale)
#dfmap['Coverage_in_percentage'] = dfmap['Coverage'].astype(str) + '%'
#fig1=px.choropleth(dfmap, locations='Country',color= 'Coverage',labels={'Coverage':'Coverage in %'},color_continuous_scale=[[0, "red"], [0.60, "yellow"],[0.8, "green"], [1, "green"]], range_color=(0,100), title='Coverage in Europe',hover_data=['Country'],scope='europe')
#pyo.plot(fig1, auto_open=True, output_type='file', filename='europemap.html')